# HW3
All functions are in a file called "functions.py" and will be included at the end.  I put them in their own file as I want to reuse the functions.

# 5.4

In [27]:
import numpy as np
from functions import *

Na =5E16
Oa = 2E-9
# as everything is in cn
Oa *= 1E2
# in 5.1 it looks like surface potentail is added
Vfb = find_Vfb(Na=Na)
print(f"bulk: {bulk(Na)} eV")
print(f"Vfb = {Vfb}  eV")

print("Part B")
Wdmax = find_Wdmax(Na)

print(f"Wdmax = {Wdmax} cm")

print("Part C")

print(f"Vt = {find_Vt(Na, Oa)} eV")

print(f"Part D")

print("If the gate was a P polly instead of an N polly, then the surface potenial would be subtracted from gate as it would lower the fermi level vs adding to the fermi level.")
print("You also need to swap the gate work functions as psi_s will be 4.05 and psi_g will be Ec-Ef/q")

print(f"Vfb = {find_Vfb(Na, N_type=False)} eV")
print(f"Vth = {find_Vt(Na, Oa, N_type=False)} eV")



bulk: 0.39862226384175575 eV
Vfb = -0.9586222638417556  eV
Part B
Wdmax = 1.4418070748537598e-05 cm
Part C
Vt = -0.09445683411809569 eV
Part D
If the gate was a P polly instead of an N polly, then the surface potenial would be subtracted from gate as it would lower the fermi level vs adding to the fermi level.
You also need to swap the gate work functions as psi_s will be 4.05 and psi_g will be Ec-Ef/q
Vfb = 0.16137773615824447 eV
Vth = 1.0255431658819045 eV


# 5.5

## part a
$\frac{dQ}{dV} = C$\
\
$\frac{4E-7}{1} = 4E-7F/cm$

## part b
$V_g = V_{fb} + \phi_s + V_{OX}$ \
$V_g - V_{fb} = \phi_s + V_{OX}$ \
$V_g - V_{fb} = \phi_B + V_{OX}$ = 1 \
$\phi_B$ and $V_{OX}$ bolth have a $N_i$ term and $\phi_B$ is needed in $V_{OX}$ meaning this is a really hard problem to solve algebraicly.  The following is a program to solve this iterivily


In [37]:
import numpy as np
from functions import *

start = 1E14
dn = 1E14 # how much to increase/decrease doping by starting
VgmVfb = 1
Cox = 4E-7

aerr = .0001
cur_vgmvfb = 0
perror = lambda E, A:(E - A)/A

Na = start
er = 1
ind = 0
print_num = 10000
# not optimal but works
while(np.abs(er) > aerr):
    cur_vgmvfb = find_Vox(Na, c=Cox) + surf_pot_t(Na)
    er = -perror(cur_vgmvfb, VgmVfb)
    Na += dn*er
    ind += 1
    if ind%print_num == 0:
        print(f"interation: {ind}")
        print(f"perror: {er}")
        print(f"current value: {cur_vgmvfb}")
        print(f"current Si doping {Na}")

print(f"actual value: {cur_vgmvfb}")
print(f"Si doping {Na}")
print(f"% error {er}")



interation: 10000
perror: 0.004327333629870811
current value: 0.9956726663701292
current Si doping 4.823479382820573e+16
interation: 20000
perror: 0.00016898064310022498
current value: 0.9998310193568998
current Si doping 4.9511880096141944e+16
actual value: 0.9999000249097983
Si doping 4.953325396729452e+16
% error 9.997509020165385e-05


so we have $N_a$ at 
$Na = 3E16cm^{-3}$
## Part C
$V_{OX} = -1 - \phi_B$

In [29]:
Vox =  -1 - bulk(Na)
print(f"Vox = {Vox}")

Vox = -1.3967694402581787


## Part D
$V_g = V_{fb} + \phi_s + V_{OX}$ \
$V_g - V_{fb} - V_{OX} = \phi_s$ \
$.5 = V_{OX} + \phi_s$ \
$.5 = \frac{\sqrt{q N_a 2 \epsilon_s \phi_s}}{C_{OX}} + \phi_s$ \
$\phi_s^2 + \frac{q N_a 2 \epsilon_s}{C_{OX}^2}\phi_s - .5^2$


In [30]:
import scipy.optimize as sci
Cox = 4E-7
Na = 3E16
poly = [1, (q*Na*2*si_di)/(Cox**2), -.5**2]
roots = np.roots(poly)
print(roots)
print(f"surf pot: {roots[1]}")


[-0.53235116  0.46961483]
surf pot: 0.4696148340105238


# Included file

In [31]:
#functions from functions.py used in this
import numpy as np
import matplotlib.pyplot as plt


# common varables
eV = 1.602E-19
q = 1.602E-19
k_J = 1.38E-23
k_eV = k_J/eV
m0 = 9.11E-31
perm = 8.85E-14 # cm
h_J = 6.63E-34
h_eV = h_J/eV
h_bar_J = 1.055E-34
h_bar_eV = h_bar_J/eV
c = 3E8
thermal_V_at_300 = (k_J*300)/q

# silicion
si_mn = .26*m0
si_mp = .39*m0
si_Eg = 1.12
si_ue = 1417
si_uh = 471
si_lattice_cosnt = 5.43
si_e_af = 4.05

# dieletrics
si_di = 11.8*perm
siO2_di = 3.9*perm
# functions
def effective_dos(m,T):
    if m == si_mn and T == 300:
        return 2.8E19
    if m == si_mp and T == 300:
        return 1.04E19
    return 2*((2*np.pi*m*k_J*T)/(h_J**2))**(3/2)

def ni(T=300,Eg = si_Eg):
    if T == 300:
        return 1E10
    Nc = effective_dos(si_mn, T)
    Nv = effective_dos(si_mp, T)
    print(T)
    return np.sqrt(Nc*Nv)*np.exp(-Eg/(2*k_eV*T))

def dos(m, EtoEcorEv):
    return (8*np.pi*m*np.sqrt(2*m*EtoEcorEv))/(h_eV**3)

def fermi(EtoEf, T=300):
    return 1/(1+np.exp(-EtoEf/(k_eV*T)))

def bulk_E(Eg, Ef, Ev):
    return (Eg/2 - (Ef-Ev))/q
 
def bulk(Na, T=300):
    return k_eV*T*np.log(Na/ni(T))

def surf_pot(Na, T=300):
    return (q*Na*Wdep(T, Na)**2)/(2*si_di)

def surf_pot_t(Na, T=300):
    return 2*bulk(Na, T)

def min_carrier(V, c, T=300):
    return c*np.exp(V/(k_eV*T))

def ex_carrier(V, c, T=300):
    min_car = min_carrier(V, T, c)
    return min_car - c

def bi_V(Na, Nd, T=300):
    if Na == 0:
        Na = ni(T)**2/Nd
    if Nd == 0:
        Nd = ni(T)**2/Na
    return k_eV*T*np.log((Na*Nd)/ni(T)**2)

def Wdep(T=300, Na=0, Nd=0):
    if Na != 0 and Nd != 0:
        return np.sqrt(((2*si_di*bi_V(Na, Nd, T))/(q))*(1/Na + 1/Nd))
    if Na != 0 and Nd == 0:
        return np.sqrt(((2*si_di*bi_V(Na, Nd, T))/(q*Na)))
    if Nd != 0 and Nd == 0:
        return np.sqrt(((2*si_di*bi_V(Na, Nd, T))/(q*Nd)))

def find_Wdmax(Na, T=300):
    return np.sqrt(((2*si_di*surf_pot_t(Na, T))/(q*Na)))

def find_Vfb(Na=0, T=300, EmEf = None, e_af = si_e_af, N_type = True):
    if EmEf == None and N_type:
        # figure 5-5 adds them
        EmEf = si_Eg/2 + bulk(Na, T)
        return e_af - (e_af + (EmEf))
    elif EmEf == None and not N_type:
        # figure #5-6 subs them
        EmEf = si_Eg/2 - bulk(Na, T)
        return (e_af + (EmEf)) - e_af
    else:
        return e_af - (e_af + (EmEf))
    

def C_dep(A, T=300, Na=0, Nd=0):
    return A*((si_di)/(Wdep(Na, Nd, T)))

def cap(di, d):
    return di/d

def find_Vt(Na, l, T=300, N_type=True):
    return find_Vfb(Na, T, N_type=N_type) + surf_pot_t(Na, T) + np.sqrt(2*q*Na*si_di*surf_pot_t(Na, T))/(cap(siO2_di,l))

def find_Vox(Na, T=300, l=0, c=None):
    if c == None:
        c = cap(siO2_di,l)
    return np.sqrt(q*Na*si_di*surf_pot_t(Na, T))/(c)

def find_Vox_Q(Qs, co=None , l=0):
    if co==None:
        co = cap(siO2_di,l)
    return Qs/co

def find_Vg(Vfb, sur_pot, Vox):
    return Vfb + sur_pot + Vox

